In [1]:
import pandas as pd
import random
from konoise import NoiseGenerator

In [2]:
file_path = "data/train.csv"
train_df = pd.read_csv(file_path)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.


In [3]:
generator = NoiseGenerator()

CSV 파일 로드 완료. 총 리뷰 수: 11263


In [4]:
# 증강 배수 설정
augmentation_factor = 5

In [5]:
# 지원하는 모든 난독화 기법
methods_list = [
    'change-vowels', 'palatalization', 'linking', 
    'liquidization', 'nasalization', 'assimilation', 'yamin-jungum'
]

In [8]:
# 데이터 증강
augmented_rows = []
for _, row in train_df.iterrows():
    original_text = row["output"]
    for _ in range(augmentation_factor):
        augmented_text = original_text
        
        # 난독화 기법을 랜덤으로 선택하여 적용
        selected_methods = random.sample(methods_list, k=random.randint(1, len(methods_list)))
        for method in selected_methods:
            augmented_text = generator.generate(augmented_text, methods=method, prob=1.)
            
            if isinstance(augmented_text, list):
                augmented_text = augmented_text[0] if augmented_text else ""

        augmented_rows.append({"ID": row["ID"], "input": augmented_text, "output": row["output"]})

데이터 증강 진행중:   0%|                                              | 2/11263 [1:20:36<7563:38:39, 2418.00s/it]


OutOfMemoryError: CUDA out of memory. Tried to allocate 728.00 MiB. GPU 0 has a total capacity of 23.64 GiB of which 663.56 MiB is free. Process 2624656 has 19.11 GiB memory in use. Process 2625065 has 3.88 GiB memory in use. Of the allocated memory 2.92 GiB is allocated by PyTorch, and 785.78 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [7]:

augmented_df = pd.DataFrame(augmented_rows)
augmented_df = pd.concat([train_df, augmented_df], ignore_index=True)
augmented_df["ID"] = [f"TRAIN_{i:06d}" for i in range(len(augmented_df))]

output_path = "aug_train_output4.csv"
augmented_df.to_csv(output_path, index=False, encoding="utf-8-sig")

증강된 데이터셋 저장 완료: augmented_train.csv
